In [4]:
import numpy as np
import sklearn
import sklearn.datasets
import theano
import theano.tensor as T
import timeit

# Use float32 as the default float data type
theano.config.floatX = 'float32'
theano.config.profile = False
theano.config.profile_memory = False

In [2]:
# Generate a dataset
np.random.seed(0)
train_X, train_y = sklearn.datasets.make_moons(200, noise=0.20)

# Size definitions
num_examples = len(train_X) # training set size
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality
nn_hdim = 3 # hiden layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = np.float32(0.01) # learning rate for gradient descent
reg_lambda = np.float32(0.01) # regularization strength 

X = theano.shared(train_X.astype('float32')) # initialized on the GPU
y = theano.shared(train_y)

W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(nn_hdim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

# Forward propagation
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2)

# The regularization term (optional)
loss_reg = 1./num_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2))) 
# the loss function we want to optimize
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg
# Returns a class prediction
prediction = T.argmax(y_hat, axis=1)

# Gradients
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

# Note that we removed the input values because we will always use the same shared variable
forward_prop = theano.function([], y_hat)
calculate_loss = theano.function([], loss)
predict = theano.function([], prediction)

gradient_step = theano.function(
    [],
    profile=True,
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

def build_model(num_passes=20000, print_loss=False):
    # Re-Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1.set_value((np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)).astype('float32'))
    b1.set_value(np.zeros(nn_hdim).astype('float32'))
    W2.set_value((np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)).astype('float32'))
    b2.set_value(np.zeros(nn_output_dim).astype('float32'))
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):
        # This will update our parameters W2, b2, W1 and b1!
        gradient_step()
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
            print "Loss after iteration %i: %f" %(i, calculate_loss())





Loss after iteration 0: 0.582899
Loss after iteration 1000: 0.327100
Loss after iteration 2000: 0.306489
Loss after iteration 3000: 0.300980
Loss after iteration 4000: 0.295883
Loss after iteration 5000: 0.289259
Loss after iteration 6000: 0.280666
Loss after iteration 7000: 0.270239
Loss after iteration 8000: 0.258530
Loss after iteration 9000: 0.246228
Loss after iteration 10000: 0.233951
Loss after iteration 11000: 0.222158
Loss after iteration 12000: 0.211124
Loss after iteration 13000: 0.200976
Loss after iteration 14000: 0.191729
Loss after iteration 15000: 0.183312
Loss after iteration 16000: 0.175607
Loss after iteration 17000: 0.168453
Loss after iteration 18000: 0.161666
Loss after iteration 19000: 0.155069


In [ ]:
# Profiling
# theano.config.profile = True
# theano.config.profile_memory = True
# gradient_step()
# print gradient_step.profile.summary()

In [3]:
%timeit build_model(print_loss=True)

Loss after iteration 0: 0.582899
Loss after iteration 1000: 0.327100
Loss after iteration 2000: 0.306489
Loss after iteration 3000: 0.300980
Loss after iteration 4000: 0.295883
Loss after iteration 5000: 0.289259
Loss after iteration 6000: 0.280666
Loss after iteration 7000: 0.270239
Loss after iteration 8000: 0.258530
Loss after iteration 9000: 0.246228
Loss after iteration 10000: 0.233951
Loss after iteration 11000: 0.222158
Loss after iteration 12000: 0.211124
Loss after iteration 13000: 0.200976
Loss after iteration 14000: 0.191729
Loss after iteration 15000: 0.183312
Loss after iteration 16000: 0.175607
Loss after iteration 17000: 0.168453
Loss after iteration 18000: 0.161666
Loss after iteration 19000: 0.155069
Loss after iteration 0: 0.582899
Loss after iteration 1000: 0.327100
Loss after iteration 2000: 0.306489
Loss after iteration 3000: 0.300980
Loss after iteration 4000: 0.295883
Loss after iteration 5000: 0.289259
Loss after iteration 6000: 0.280666
Loss after iteration 700